In [1]:
import pandas as pd
import pickle
import numpy as np
from brainflow.board_shim import BoardShim, BrainFlowInputParams, LogLevels, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes
import matplotlib
import os
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset
from torchvision import models
from pytorch_lightning.loggers import WandbLogger
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy
from pytorch_lightning.callbacks import Callback
import torchinfo
import cv2
from torch.utils.data import DataLoader
import wandb

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
%matplotlib inline
scaler = StandardScaler()

In [2]:
wandb.login()
api_key = 'a45abb01f9556b57620ce77c8984452bee7a8772'
board_id = 38
sf = 256

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [3]:
class PickleLatentDatasetLoader(Dataset):
    def __init__(self, annotations_file, dir):
        self.file_lists = pd.read_csv(annotations_file, header=None)
        self.dir = dir

    def __len__(self):
        return len(self.file_lists)

    def __getitem__(self, idx):
        file_path = os.path.join(self.dir, self.file_lists.iloc[idx, 0].replace('/','\\'))
        label = self.file_lists.iloc[idx, 1]

        pkl_file = open(file_path, 'rb')
        results = pickle.load(pkl_file)
        results = np.array(results)
        pkl_file.close()

        results = torch.tensor(results)
        label = torch.tensor(label).type(torch.FloatTensor)

        return (results[0].squeeze(), results[1].squeeze(), label)

In [4]:
pkl_dir = 'D:\\Nicko\\TUGAS_AKHIR\\Dataset_new\\Dataset_TA_pkl\\LATENT_32640_2CHANNEL\\'
path_file = pkl_dir + 'path_and_label.csv'
train_dir = pkl_dir + 'train_set.csv'
valid_dir = pkl_dir + 'valid_set.csv'
test_dir = pkl_dir + 'test_set.csv'

In [5]:
datasets_train = PickleLatentDatasetLoader(
    annotations_file=train_dir,
    dir=pkl_dir
)
datasets_valid = PickleLatentDatasetLoader(
    annotations_file=valid_dir,
    dir=pkl_dir
)
datasets_test = PickleLatentDatasetLoader(
    annotations_file=test_dir,
    dir=pkl_dir
)

In [6]:
latent_loaded = datasets_train[0]

In [7]:
latent_loaded[0].shape

torch.Size([64, 10, 51])

In [8]:
train_dataloader = DataLoader(datasets_train, batch_size=10)
validation_dataloader = DataLoader(datasets_valid, batch_size=10)
test_dataloader = DataLoader(datasets_test, batch_size=10)

In [11]:
class LitClassifierWithResnet50(pl.LightningModule):
    def __init__(self, input_shape, unfreeze_layer=None,learning_rate=1e-3):
        super().__init__()

        # log hyperparameters
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        self.loss_fn = torch.nn.BCEWithLogitsLoss()

        # variable
        self.predictions = np.array([])
        self.targets = np.array([])
        self.predictions_val = np.array([])
        self.targets_val = np.array([])
        self.predictions_test = np.array([])
        self.targets_test = np.array([])
        self.f1_fn = BinaryF1Score()
        self.acc_fn = BinaryAccuracy()
        self.f1_train = 0
        self.f1_val = 0
        self.f1_test = 0

        # Convolutional
        self.weights = models.ResNet50_Weights.DEFAULT
        self.backbone = models.resnet50(weights=self.weights)
        self.unfreeze_layer = unfreeze_layer
        self._change_layer()

    def _change_layer(self):
        # UNFREEZE LAYER IF MEET NAME BELOW
        print(self.backbone.fc)
        name = self.unfreeze_layer
        for param_name, param in self.backbone.named_parameters():
            trainable = False
            if name:
                for train_name in name:
                    if train_name in param_name:
                        trainable = True
                        break
                param.requires_grad = trainable
            else:
                param.requires_grad = False

        # change input layer
        self.backbone.conv1 = torch.nn.Conv2d(128, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

        # change last layer
        self.backbone.fc = torch.nn.Linear(in_features=2048, out_features=1, bias=True)

        # model summary
        torchinfo.summary(model=self.backbone,
                          input_size=(10,128,10,51),
                          col_names=["input_size", "output_size", "num_params", "trainable"],
                          col_width=20,
                          row_settings=["var_names"]
        )

    def forward(self, x1, x2):
         x_combined = torch.cat([x1,x2], dim=1)
         y_logits = self.backbone(x_combined)
         y_pred = torch.round(torch.sigmoid(y_logits))
         return y_pred

    def training_step(self, batch, batch_idx):
        x1, x2, y = batch
        x_combined = torch.cat([x1,x2], dim=1)
        y_logits = self.backbone(x_combined)
        y_logits = y_logits.squeeze()
        loss = self.loss_fn(y_logits, y)
        self.log("train_loss", loss)
        y_preds = torch.round(torch.sigmoid(y_logits.cpu())).detach().numpy()
        self.predictions = np.append(self.predictions, y_preds, axis=0)
        self.targets = np.append(self.targets, y.cpu(), axis=0)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x1, x2, y = val_batch
        x_combined = torch.cat([x1,x2], dim=1)
        y_logits = self.backbone(x_combined)
        y_logits = y_logits.squeeze()
        val_loss = self.loss_fn(y_logits, y)
        self.log("val_loss", val_loss)
        y_preds = torch.round(torch.sigmoid(y_logits.cpu())).detach().numpy()
        self.predictions_val = np.append(self.predictions_val, y_preds, axis=0)
        self.targets_val = np.append(self.targets_val, y.cpu(), axis=0)

    def test_each_epoch(self):
        self.predictions_test = np.array([])
        self.targets_test = np.array([])
        self.eval()
        with torch.no_grad():
            for x1,x2,y in test_dataloader:
                x_combined = torch.cat([x1,x2], dim=1)
                x_combined = x_combined.to("cuda")
                y_logits = self.backbone(x_combined)
                y_logits = y_logits.squeeze()
                test_epoch_loss = self.loss_fn(y_logits, y.to("cuda"))
                y_preds = torch.round(torch.sigmoid(y_logits.cpu())).detach().numpy()
                self.predictions_test = np.append(self.predictions_test, y_preds, axis=0)
                self.targets_test = np.append(self.targets_test, y.cpu(), axis=0)
            self.f1_test = self.f1_fn(torch.tensor(self.predictions_test), torch.tensor(self.targets_test))
            self.log('F1_Score_test', self.f1_test)
            self.log("test_epoch_loss", test_epoch_loss)

    def test_step(self, test_batch, batch_idx):
        x1, x2, y = test_batch
        x_combined = torch.cat([x1,x2], dim=1)
        y_logits = self.backbone(x_combined)
        y_logits = y_logits.squeeze()
        test_loss = self.loss_fn(y_logits, y)
        test_acc = self.acc_fn(y_logits, y)
        self.log("test_loss", test_loss)
        y_preds = torch.round(torch.sigmoid(y_logits.cpu())).detach().numpy()
        self.log("test_loss", test_loss)
        self.log("test_acc", test_acc)
        return {"test_loss": test_loss, "test_acc": test_acc}

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=self.learning_rate)
        return optimizer

In [ ]:
class LoggingCallback(Callback):
    def __init__(self):
        super().__init__()
        self.f1 = BinaryF1Score()

    def on_train_epoch_start(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule") -> None:
        pl_module.predictions = np.array([])
        pl_module.targets = np.array([])

    def on_train_epoch_end(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule") -> None:
        pl_module.f1_train = pl_module.f1_fn(torch.tensor(pl_module.predictions), torch.tensor(pl_module.targets))
        pl_module.log('F1_Score_train', pl_module.f1_train)

    def on_validation_epoch_start(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule") -> None:
        pl_module.predictions_val = np.array([])
        pl_module.targets_val = np.array([])

    def on_validation_epoch_end(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule") -> None:
        pl_module.f1_val = pl_module.f1_fn(torch.tensor(pl_module.predictions_val), torch.tensor(pl_module.targets_val))
        pl_module.log('F1_Score_val', pl_module.f1_val)
        pl_module.test_each_epoch()
        f1_avg = torch.mean(torch.tensor([pl_module.f1_train,pl_module.f1_val,pl_module.f1_test]))
        pl_module.log('F1_Score_avg', f1_avg)


In [ ]:
save_dir = 'D:\\Nicko\\TUGAS_AKHIR\\Classifier\\model_4\\'
run_name = 'Run_9_allfreeze_new_data'

In [ ]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor="F1_Score_avg", mode="max", dirpath=save_dir+"classifier\\"+run_name,
    filename="classifier-{epoch:02d}-{F1_Score_avg:.2f}",)
logging_callback = LoggingCallback()

In [ ]:
wandb_logger = WandbLogger(project='classifier_resnet50', name=run_name, save_dir=save_dir)

In [12]:
pl.seed_everything(42, workers=True)
classifier = LitClassifierWithResnet50(input_shape=(128,10,51), learning_rate=1e-3, unfreeze_layer=None)

Global seed set to 42


Linear(in_features=2048, out_features=1000, bias=True)


In [ ]:
trainer = pl.Trainer(max_epochs=10000, devices=1, accelerator='gpu', log_every_n_steps=9, logger=wandb_logger, callbacks=[checkpoint_callback, logging_callback])
trainer.fit(classifier, train_dataloader, validation_dataloader)
wandb.finish()

In [ ]:
trainer.test(classifier, test_dataloader)

MODEL LOADING

In [10]:
classifier_loaded = LitClassifierWithResnet50(input_shape=(128,10,51), unfreeze_layer=['layer4']).load_from_checkpoint('D:\\Nicko\\TUGAS_AKHIR\\Classifier\\model_4\\classifier\\run_8_layer4train\\classifier-epoch=4543-F1_Score_avg=0.84.ckpt')

In [13]:
trainer = pl.Trainer(max_epochs=50000, devices=1, accelerator='gpu', log_every_n_steps=9)
trainer.test(classifier_loaded, test_dataloader)

D:\Nicko\TUGAS_AKHIR\TA_env\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.4444444477558136
        test_loss           1.3812031745910645
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.3812031745910645, 'test_acc': 0.4444444477558136}]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.4444444477558136
        test_loss           1.3812031745910645
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.3812031745910645, 'test_acc': 0.4444444477558136}]